In [10]:
import numpy as np
import cv2
frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10,150)
widthImg=540
heightImg =640

In [11]:
def preprocessing (frame):
    frameGray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    frameBulr = cv2.GaussianBlur(frameGray,(5,5),1)
    frameCanny = cv2.Canny(frameBulr,100,100)
    
    kernel = np.ones((5,5))
    frameDilate = cv2.dilate(frameCanny,kernel,iterations = 2)
    frameErosion = cv2.erode(frameDilate,kernel,iterations=1)
    preprocessedFrame = frameErosion
    return preprocessedFrame
    

In [12]:
def getContours(frame):
    biggest = np.array([])
    maxArea = 0
    contours,hierarchy = cv2.findContours(frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>5000:
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)
            if area >maxArea and len(approx) == 4:
                biggest = approx
                maxArea = area
    cv2.drawContours(frameContour, biggest, -1, (255, 0, 0), 20)
    return biggest

In [13]:
def reorder (myPoints):
    myPoints = myPoints.reshape((4,2))
    myPointsNew = np.zeros((4,1,2),np.int32)
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    diff = np.diff(myPoints,axis=1)
    myPointsNew[1]= myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    return myPointsNew
 
def getWarp(frame,biggest):
    biggest = reorder(biggest)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0, 0], [widthframe, 0], [0, heightframe], [widthframe, heightframe]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    frameOutput = cv2.warpPerspective(img, matrix, (widthImg, heightImg))
 
    frameCropped = frameOutput[20:framegOutput.shape[0]-20,20:frameOutput.shape[1]-20]
    frameCropped = cv2.resize(frameCropped,(widthframe,heightframe))
 
    return frameCropped

In [14]:
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

In [ ]:
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame,(widthImg,heightImg))
    frameContour = frame.copy()
 
    preprocessedFrame = preprocessing(frame)
    biggest = getContours(preprocessedFrame)
    if biggest.size !=0:
        frameWarped=getWarp(frame,biggest)

        imageArray = ([frameContour, frameWarped])
        cv2.imshow("ImageWarped", frameWarped)
    else:
        imageArray = ([frameContour, frame])
 
    stackedImages = stackImages(0.6,imageArray)
    cv2.imshow("WorkFlow", stackedImages)
 
    if cv2.waitKey(1) and 0xFF == ord('q'):
        break